# Worflow of this notebook:

## Input
- single channel `tif` files 
- a `Log.txt` file generated via RS-FISH Fiji plugin for each channel in your images 
- an `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

## Outpout
- folder containing a `csv` file for each provioded `tif`
- `merge.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

# 0) Imports and functions

The functions required for this to work are collected in the `pipelines/fish_utils` folder. Download the folder from `/../` in this repository and `sys.path.append(/path/to/fish_utils/)`.

In [ ]:
import json
import os
from glob import glob
import sys
import tifffile
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from natsort import natsorted
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from skimage.io import imread

from utils.spot_detection import read_parameters, make_fiji_command, detect_spots, combine_csv, add_sample_info, create_folder, plot_detections

# 1) Detect spots 
Before running this, open a few of your images in fiji, run RS-FISH on it, and save the Log.txt file with the spot detection parameters for each channel - **IMPORTAINT!**

In [ ]:
in_path = "/home/stumberger/ep2024/example/" # path to upper level directory
tif_subfolder = "tif/" # where the tifs which to detect spots in are saved
out_subfolder = "detections" # what to name the subfolder ot save the spots
channels = [1,2] # which channel to detect spots in (counting starts at 0!)
detection_settings  = None

# link to your fiji installation with RS-FISH installed
fiji_path = "/home/stumberger/tools/Fiji.app/ImageJ-linux64"

# link to the Fiji Macro (under ".../subscripts/utils/RS_macro_param.ijm")
macro_path =  "/home/stumberger/ep2024/subscripts/utils/RS_macro_param.ijm"

In [ ]:
# generate default detection_settings path if none given
if detection_settings is None:
    detection_settings = []
    for c in channels:
        detection_settings.append(os.path.normpath(f'{in_path}/{out_subfolder}/ch{c}.txt'))
else:
    pass

# run
detect_spots(in_path,detection_settings,channels,
             tif_subfolder = tif_subfolder,
             out_subfolder = out_subfolder,
             macro_path =macro_path,
             fiji_path = fiji_path)

# combine all csvs in a folder into 1 and add sample info
combine_csv(in_path,tif_subfolder,out_subfolder)
add_sample_info(in_path, info=f"{in_path}/acquisition_info.json",out_subfolder=out_subfolder)

# 2) Visualize detection

In [ ]:
 #path to `merge.csv` file; if None defaults to /.../detections/merge.csv 
path_spots= os.path.normpath(f'{in_path}/{out_subfolder}/merge.csv') 

#where to save visualisations; if None defaults to /.../detections/vis
out_folder = os.path.normpath(f'{in_path}/{out_subfolder}/vis/') 

# which channel to plot spots for (counting starts at 0!)
channels = channels 

# intensity scaling for images
range_quantiles = (0.03, 0.99)

plot_detections(in_path,channels,path_spots=path_spots,tif_subfolder=tif_subfolder,out_folder=out_folder)